In [5]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'OSMSD.db'
conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()

cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()

cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()
# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
cur.execute('''
    CREATE TABLE nodes(id INTEGER, lat NUM, lon NUM, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TIMESTAMP)
''')
cur.execute('''
    CREATE TABLE ways(id INTEGER, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TIMESTAMP)
''')
cur.execute('''
    CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
cur.execute('''
    CREATE TABLE ways_nodes(id INTEGER, node_id real, position INTEGER)
''')

# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'].decode("utf-8"), i['type']) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)

with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['lat'], i['lon'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
cur.executemany("INSERT INTO nodes(id,lat,lon,user,uid,version,changeset,timestamp) values (?, ?, ?, ?, ?, ?, ?, ?);", to_db)

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)

with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'].decode("utf-8"), i['type']) for i in dr]
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)

with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)

# commit the changes
conn.commit()

cur.execute('SELECT * FROM nodes_tags limit 10')
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

cur.execute('SELECT * FROM nodes limit 10')
all_rows = cur.fetchall()
print('2):')
pprint(all_rows)

cur.execute('SELECT * FROM ways limit 10')
all_rows = cur.fetchall()
print('3):')
pprint(all_rows)

cur.execute('SELECT * FROM ways_tags limit 10')
all_rows = cur.fetchall()
print('4):')
pprint(all_rows)

cur.execute('SELECT * FROM ways_nodes limit 10')
all_rows = cur.fetchall()
print('5):')
pprint(all_rows)


1):
[(28828453, u'created_by', u'JOSM', u'regular'),
 (28828658, u'created_by', u'JOSM', u'regular'),
 (31526883, u'barrier', u'gate', u'regular'),
 (48857474, u'highway', u'turning_circle', u'regular'),
 (48857638, u'name', u'12th and Imperial Transit Center', u'regular'),
 (48857661, u'railway', u'level_crossing', u'regular'),
 (48857672, u'railway', u'level_crossing', u'regular'),
 (48857761, u'ref', u'10', u'regular'),
 (48857762, u'source', u'Bing', u'regular'),
 (48857793, u'source', u'Bing', u'regular')]
2):
[(28828453,
  32.7642168,
  -117.2332461,
  u'KindredCoda',
  14293,
  4,
  551611,
  u'2008-07-20T10:38:50Z'),
 (28828658,
  32.7642096,
  -117.2326882,
  u'KindredCoda',
  14293,
  5,
  551611,
  u'2008-07-20T10:38:58Z'),
 (31526883,
  32.7667756,
  -117.2309013,
  u'Brian@Brea',
  416346,
  5,
  27354905,
  u'2014-12-09T11:57:06Z'),
 (48857474,
  32.827606,
  -117.223999,
  u'Sat',
  48060,
  3,
  8107292,
  u'2011-05-10T21:56:42Z'),
 (48857638,
  32.7057663,
  -117.15317

In [11]:
cur.execute ('SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 1;
')

SyntaxError: EOL while scanning string literal (<ipython-input-11-6096c741a902>, line 1)